In [1]:
import numpy as np

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
from time import ctime
import plotly_express as plx
from tqdm.notebook import tqdm

from sys import path as syspath
import pickle
from sys import exc_info

In [2]:
ip = 0
allpickles = []
for cd, ds, fs in os.walk("/data"):
    for f in fs:
        if f.endswith("pkl"):
            path = os.path.join(cd, f)
            with open(path,"rb") as fh:
                peek = fh.peek()[:30]
                if b"Regions" not in peek:
                    continue
            allpickles += [(path, ctime(os.path.getmtime(path)), os.path.getsize(path)/1e6, peek)]
#             ip += 1
#     if ip>1000:
#         break

In [3]:
allpickles = pd.DataFrame(allpickles,columns=["path","raw time","size(MB)","peek"])

In [5]:
# syspath.append("/home/jupyter-srdjan/srdjan_functs/")
# from islets.Regions import load_regions
# from islets.utils import saveRois
# ii = np.random.permutation(allpickles.index[:])
# for ix in tqdm(ii):
#     peek = allpickles.loc[ix,"peek"]
#     if b'islets.Regions\nRegions' in peek:
#         allpickles.loc[ix,"success"] = np.nan
#         continue
#     try:
#         path = allpickles.loc[ix,"path"]
#         regions = load_regions(path,
#                                baremin=True,
#                                mergeSizeTh=5,
#                                calcInterest=False,
#                               )
#         saveRois(regions,
#                  os.path.split(path)[0],
#                  os.path.split(path)[1].split("_rois.pkl")[0],
#                  add_date=False,
#                 )
#         allpickles.loc[ix,"success"] = 1
#     except:
#         allpickles.loc[ix,"success"] = 0
#         print (f"Could not recast {path}")

In [8]:
allpickles.peek.value_counts()

b'\x80\x03cislets.Regions\nRegions\nq\x00)\x81'          9877
b'\x80\x03cRegions1\nRegions\nq\x00)\x81q\x01}q\x02('      88
b'\x80\x03cislets.Regions1\nRegions\nq\x00)'                7
b'\x80\x03c__main__\nRegions\nq\x00)\x81q\x01}q\x02('       1
Name: peek, dtype: int64

In [6]:
correctBeginning = allpickles.peek.value_counts().sort_values().index[-1]

In [7]:
remaining = allpickles[allpickles["peek"]!=correctBeginning].copy()

In [8]:
syspath.append("/home/jupyter-srdjan/srdjan_functs/tmp/")

In [9]:
syspath.append("/home/jupyter-srdjan/srdjan_functs/")

In [14]:
remaining.peek.value_counts()

b'\x80\x03cRegions1\nRegions\nq\x00)\x81q\x01}q\x02('    88
b'\x80\x03cislets.Regions1\nRegions\nq\x00)'              7
b'\x80\x03c__main__\nRegions\nq\x00)\x81q\x01}q\x02('     1
Name: peek, dtype: int64

In [16]:
singleOutlier = remaining.peek.value_counts().index[-1]
singleOutlier

b'\x80\x03c__main__\nRegions\nq\x00)\x81q\x01}q\x02('

In [23]:
pathToOutlier = remaining[remaining.peek==singleOutlier].iloc[0].path
os.remove(pathToOutlier)
assert not os.path.isfile(pathToOutlier)

In [30]:
remaining = remaining[remaining.peek!=singleOutlier]

In [11]:
remaining.peek.value_counts()

b'\x80\x03cRegions1\nRegions\nq\x00)\x81q\x01}q\x02('    19
b'\x80\x03cislets.Regions1\nRegions\nq\x00)'              7
Name: peek, dtype: int64

In [12]:
mostAbundant = remaining.peek.value_counts().index[0]
mostAbundant

b'\x80\x03cRegions1\nRegions\nq\x00)\x81q\x01}q\x02('

In [13]:
from islets.Regions import Regions
from islets.utils import saveRois

In [14]:
done = []

In [15]:
for ix in tqdm(remaining.index):
    row = remaining.loc[ix]
    if row.peek!=mostAbundant: continue
    if ix in done: continue
    path = row.path
    try:
        with open(path,"rb") as f:
            regions = pickle.load(f)
        regions = Regions(regions)
        regions.update()
        saveRois(regions,
                 os.path.split(path)[0],
                 os.path.split(path)[1].split("_rois.pkl")[0],
                 add_date=False,
                )
        done += [ix]
    except:
        print ("did not work for", path)
        print (str(exc_info()))

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/tljh/user/envs/physio/lib/python3.7/site-packages/tensorflow/pyth

In [16]:
remaining.drop(index=done, inplace=True)

In [17]:
remaining.peek.value_counts()

b'\x80\x03cislets.Regions1\nRegions\nq\x00)'    7
Name: peek, dtype: int64

In [19]:
done = []

In [24]:
for ix in tqdm(remaining.index):
    row = remaining.loc[ix]
    if ix in done: continue
    path = row.path
    try:
        with open(path,"rb") as f:
            regions = pickle.load(f)
        regions = Regions(regions)
        regions.update()
        saveRois(regions,
                 os.path.split(path)[0],
                 os.path.split(path)[1].split("_rois.pkl")[0],
                 add_date=False,
                )
        done += [ix]
    except:
        print ("did not work for", path)
        print (str(exc_info()))
        

In [25]:
remaining.drop(index=done, inplace=True)

In [26]:
remaining

,path,raw time,size(MB),peek
